In [ ]:

import httpx
import json
from joblib import Memory
from bs4 import BeautifulSoup
memory = Memory("./cache", verbose=0)

@memory.cache
def fetch_data(url : str) -> str:
    
    cookies = {
        'req-device-token': '7de4109a-0e61-4488-b544-b451d2321403',
        'cf_clearance': 'wLDXjJ.KdsclQ7VCV72DxifaOwmOoKWRiKFauzNLulo-1766670311-1.2.1.1-estcFeioX18Hn3NLEbhu3_cN3NEMyW6HZ4cIbs9DdErQX2HnejhNNcuah8dHPiaMig.n0bHutzNz12u9kxluSJDyqadLqIQtPAW.x5iL9mu8E3YT3CZxLG_mQRqhJ25Yn.K6PT_EfgpT4y65biIg34WnurMVcLhe0lNE0hxPLUWr8JnUqbx5UyOclmQNBlBluOKK0u.OgNX8oAvRgOFdf9KHK2SvZAafGKaz4jbbf34',
        '__cf_bm': 'L2zIqLFZrqYT3yludtz_cx3IAmkqGIZNFYALmaicyEk-1766670979-1.0.1.1-ebldrairypvv6uBJWyogayDUj2bOcz.TJMAJ0CdDRrOx8bW.lVU95FldiIQPIU_Zsfw9E3F_Vzuhj2dwUduWNA90jONogjXrIX80drruYzE',
    }

    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-US,en;q=0.9,ar;q=0.8',
        'cache-control': 'no-cache',
        'pragma': 'no-cache',
        'priority': 'u=0, i',
        'referer': 'https://duckduckgo.com/',
        'sec-ch-ua': '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Linux"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36',
        # 'cookie': 'req-device-token=7de4109a-0e61-4488-b544-b451d2321403; cf_clearance=wLDXjJ.KdsclQ7VCV72DxifaOwmOoKWRiKFauzNLulo-1766670311-1.2.1.1-estcFeioX18Hn3NLEbhu3_cN3NEMyW6HZ4cIbs9DdErQX2HnejhNNcuah8dHPiaMig.n0bHutzNz12u9kxluSJDyqadLqIQtPAW.x5iL9mu8E3YT3CZxLG_mQRqhJ25Yn.K6PT_EfgpT4y65biIg34WnurMVcLhe0lNE0hxPLUWr8JnUqbx5UyOclmQNBlBluOKK0u.OgNX8oAvRgOFdf9KHK2SvZAafGKaz4jbbf34; __cf_bm=L2zIqLFZrqYT3yludtz_cx3IAmkqGIZNFYALmaicyEk-1766670979-1.0.1.1-ebldrairypvv6uBJWyogayDUj2bOcz.TJMAJ0CdDRrOx8bW.lVU95FldiIQPIU_Zsfw9E3F_Vzuhj2dwUduWNA90jONogjXrIX80drruYzE',
    }

    response = httpx.get(url,cookies=cookies, headers=headers,timeout=30,follow_redirects=True)
    textof=response.text
    
    assert not "you have been blocked" in textof.lower(), "Blocked by the website"
    return textof

def get_all_pages(url: str) -> list[str]:
    all_pages = []
    page_number = 1

    while True:
        paged_url = f"{url}/{page_number}"
        print(f"Fetching page {page_number}: {paged_url}")
        page_content = fetch_data(paged_url)
        if "لا توجد نتائج" in page_content:
            break
        all_pages.append(page_content)
        page_number += 1

    return all_pages



def parse_page(page:str)-> list[dict]:
    output=[]
    soup = BeautifulSoup(page, "html.parser")
    listings = soup.select("#__next > main > div > div._root__Szbd6 > div > div:nth-child(2) > div._container__Lu67A > div._list__Ka30R > div")    
    for listing in listings:
        def get_text(selector):
            element = listing.select_one(selector)
            return element.text.strip() if element else None
        def price_or_auction(strornum):
            try:
                result = float(strornum)
                return result
            except (ValueError, TypeError):
                dict_item["is_auction"]=True
                return None
        
        dict_item={}
        dict_item["title"]=get_text("a > div > div._content__W4gas > div._titleRow__1AWv1 > h4")
        dict_item["is_auction"]=False
        dict_item["url"]= listing.select_one("a")['href'] if listing.select_one("a") else None
        dict_item["price"]=price_or_auction(get_text("a > div > div._content__W4gas > div._price__X51mi > span"))
        dict_item["description"]=get_text("a > div > div._content__W4gas > div._description__zVaD6")
        dict_item["city"]=get_text("a > div > div._content__W4gas > div._footer__CnldH > p > span:nth-child(1)")
        dict_item["neighborhood"]=get_text("a > div > div._content__W4gas > div._footer__CnldH > p > span:nth-child(2)").replace("-","").strip()
        svgimgs = listing.select("a > div > div._content__W4gas > div._specs__nbsgm .icon_icon___L1OO img[src]")
        svgurl2span = {
            img['src']: img.parent
            for img in svgimgs
        }
        for k, span in svgurl2span.items():
            if not k:
                continue
            icon_name :str= k.split('/')[-1].split('.')[0]
            value: str | None = span.parent.get_text(strip=True)
                
            dict_item[icon_name] = value
        output.append(dict_item)
    return output
    

    
    
    
request = fetch_data("https://sa.aqar.fm/%D8%B9%D9%82%D8%A7%D8%B1%D8%A7%D8%AA/")
soup = BeautifulSoup(request, "html.parser")
out=parse_page(request)
with open('data.html', 'w', encoding='utf-8') as f:
    f.write(request)
json.dump(out,open("output.json","w"),ensure_ascii=False,indent=4)


[{'title': 'دور للبيع في شارع شهيد الدين ثم الوطن فهد إبراهيم علي أب, حي النرجس, مدينة الرياض, منطقة الرياض',
  'price': '1,500,000',
  'description': 'مشروع ادوار سِـــدرا مشروع يتميز بموقعه الاستراتجي بحي النرجس جنوب طريق انس بن مالك وتصاميمه الفريدة.\n\nدور ثاني بمساحة 169م\n\nمحتويات الدور:\n-غرفة نوم ماستر \n-غرفة نوم 1\n-غرفة نوم 2\n-غرفة خادمة بدورة مياه\n-صالة\n-مطبخ\n-سطح امامي \n-بلكونة عند الغرفة الماستر\n-3 دورات مياه',
  'city': 'الرياض',
  'neighborhood': 'حي النرجس',
  'area': '116 م²',
  'bed-king': '4',
  'bath': '3'},
 {'title': 'فيلا للبيع في شارع عذبة, حي حطين, مدينة الرياض, منطقة الرياض',
  'price': '4,900,000',
  'description': '\u200fللبيع فلتين مودرن بحي حطين بتصميم مميز والاكثر شراحة \nالمساحة:  300م \nشارع 20م شمالي \n\nفيلا 6 غرف نوم مع ملحق ب 4.9 مليون\nفيلا 4 غرف نوم بدون ملحق ب 4.7 مليون\n\nالتفاصيل : \nالارضي: ملحق خارجي - موقفين سيارات - مجلس - صالة طعام - صالة عائلية - مطبخ ومستودع وركن الغسيل وغرفة خادمة - فناء خلفي للصالة - مؤسس مصعد وراكب تكييف مخفي 